In [1]:
import numpy as np
import pylab as pl
from scipy.integrate import odeint
from matplotlib import animation
import random
%matplotlib inline

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import arrayfire as af
af.set_backend('opencl')

In [3]:
pl.rcParams['figure.figsize'] = 12, 7.5 
pl.rcParams['figure.dpi'] = 100 
pl.rcParams['image.cmap'] = 'jet' 
pl.rcParams['lines.linewidth'] = 1.5 
pl.rcParams['font.family'] = 'serif' 
pl.rcParams['font.weight'] = 'bold' 
pl.rcParams['font.size'] = 20 
pl.rcParams['font.sans-serif'] = 'serif' 
pl.rcParams['text.usetex'] = False 
pl.rcParams['axes.linewidth'] = 1.5 
pl.rcParams['axes.titlesize'] = 'medium' 
pl.rcParams['axes.labelsize'] = 'medium' 
pl.rcParams['xtick.major.size'] = 8 
pl.rcParams['xtick.minor.size'] = 4 
pl.rcParams['xtick.major.pad'] = 8 
pl.rcParams['xtick.minor.pad'] = 8 
pl.rcParams['xtick.color'] = 'k' 
pl.rcParams['xtick.labelsize'] = 'medium' 
pl.rcParams['xtick.direction'] = 'in' 
pl.rcParams['ytick.major.size'] = 8 
pl.rcParams['ytick.minor.size'] = 4 
pl.rcParams['ytick.major.pad'] = 8 
pl.rcParams['ytick.minor.pad'] = 8 
pl.rcParams['ytick.color'] = 'k' 
pl.rcParams['ytick.labelsize'] = 'medium' 
pl.rcParams['ytick.direction'] = 'in'

In [4]:
def lagrange_basis_coeffs( X ):
    '''
    Returns a matrix of coefficients of the
    lagrange basis polynomials
    '''
    
    denominator  = np.ones([X.size])
    null = np.zeros([X.size])
    
    for j in range(X.size):
        k = np.poly1d([1])
        for m in range(X.size):
            if m != j:
                denominator[j] *= (X[j] - X[m])
                k *= np.poly1d([1, -X[m]])
        coeff = (k.c) / denominator[j]
        null  = np.vstack((null, coeff))
        
    basis_coeffs = np.delete(null, (0), axis = 0)
    
    '''
    Global Variables
    '''
    
    
    coeff_array = af.interop.np_to_af_array(lagrange_basis_coeffs(X))
    
    Y = af.interop.np_to_af_array(X)
    
    
    return ()

In [5]:
def linspace(start, end, number_of_points):
    '''
    Linspace implementation using arrayfire.
    '''
    X = af.range(number_of_points)
    d = (end - start) / (number_of_points - 1)
    X = X * d
    X = X + start
    return X


In [6]:
def LagrangeBasisFnsValues(i, x):
    '''
    Returns the value of lagrange polynomial
    at index i for points in x
    '''
    
    
    
    coeff       = af.arith.cast(af.transpose(coeff_array[i]), af.Dtype.f32)
    outer_prod  = af.blas.matmul(coeff, x)
    x_tile      = af.tile(x,coeff.shape[0])
    power       = linspace((Y.shape[0] - 2), -1, Y.shape[0])
    powtile     = af.tile(power, 1, x.shape[1])
    x_pow       = af.arith.pow(x_tile, powtile)
    res         = outer_prod * x_pow
    lagrange    = af.algorithm.sum(res, dim=0)
    
    return(lagrange)

In [10]:
X    = np.linspace(-1, 1, 5, dtype = float)
x    = af.randu(1, 10)
vals = LagrangeBasisFnsValues(X, 2, x)
print(vals)

arrayfire.Array()
Type: float

[1 10 1 1]
   -0.2961    -0.5224    -0.5587     0.9422     0.6516    -0.2382     0.1417     0.9160     0.9009    -0.5426 



In [11]:
def fn(x):
    return(4 * (x ** 4) - 5 * (x ** 2) + 1)
    

In [12]:
print(fn(x) - vals)

arrayfire.Array()
Type: float

[1 10 1 1]
    0.0000     0.0000     0.0000     0.0000     0.0000     0.0000    -0.0000     0.0000     0.0000     0.0000 



In [10]:
a = af.randu(1, 2)
b = af.randu(1, 2)
print(a, b, a - b)

arrayfire.Array()
Type: float

[1 2 1 1]
    0.9397     0.7245 
 arrayfire.Array()
Type: float

[1 2 1 1]
    0.8098     0.1080 
 arrayfire.Array()
Type: float

[1 2 1 1]
    0.1299     0.6165 



In [11]:
x = af.transpose(af.Array([7, 6, 2, 3]))
power = af.Array([0, 1, 2, 3, 4])
powtile = af.tile(power, 1, 4)
xtile = af.tile(x, 5)
res = af.arith.pow(xtile, powtile)
print(powtile, xtile, res)

arrayfire.Array()
Type: float

[5 4 1 1]
    0.0000     0.0000     0.0000     0.0000 
    1.0000     1.0000     1.0000     1.0000 
    2.0000     2.0000     2.0000     2.0000 
    3.0000     3.0000     3.0000     3.0000 
    4.0000     4.0000     4.0000     4.0000 

 arrayfire.Array()
Type: float

[5 4 1 1]
    7.0000     6.0000     2.0000     3.0000 
    7.0000     6.0000     2.0000     3.0000 
    7.0000     6.0000     2.0000     3.0000 
    7.0000     6.0000     2.0000     3.0000 
    7.0000     6.0000     2.0000     3.0000 

 arrayfire.Array()
Type: float

[5 4 1 1]
    1.0000     1.0000     1.0000     1.0000 
    7.0000     6.0000     2.0000     3.0000 
   49.0000    36.0000     4.0000     9.0000 
  343.0000   216.0000     8.0000    27.0000 
 2401.0000  1296.0000    16.0000    81.0000 


